### Imports

In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.environ.get('base_path'))

In [4]:
%load_ext autoreload
%autoreload 2
    
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tenacity import retry, stop_after_attempt, wait_exponential

from models.GPT_4.openai_utils import get_gpt_tokens
from models.GPT_4.openai_utils import get_batch_completion_tasks
from models.GPT_4.openai_utils import generate_jsonl_file
from models.GPT_4.openai_utils import upload_file_to_openai
from models.GPT_4.openai_utils import create_batch_completion_job
from models.GPT_4.openai_utils import get_batch_job_status
from models.GPT_4.openai_utils import get_batch_job_results
from models.GPT_4.openai_utils import save_batch_results_to_jsonl
from models.GPT_4.openai_utils import map_jsonl_batch_completion_results_to_df
from models.GPT_4.openai_utils import generate_openai_fine_tuning_json_from_df
from models.GPT_4.openai_utils import upload_fine_tuning_file_to_openai
from models.GPT_4.openai_utils import fine_tune_gpt_model
from models.GPT_4.openai_utils import check_fine_tuning_job_status
from models.GPT_4.openai_utils import get_fine_tuned_model_details

from utils.dataframe_utils import read_pandas_csv_clean_columns_names
from utils.dataframe_utils import get_model_cross_validation

from utils.text_utils import remove_pii
from utils.text_utils import lang_code_to_nltk
from utils.text_utils import iso_639_1_code_language_map

from utils.utils import find_file

import torch

from collections import Counter
from pprint import pprint
import ast
import time
import json
import re
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
dataset_path = find_file('checkpoint_sales_files_embedding.pkl')
print(dataset_path)

/Users/nathaniel.maymon/Downloads/Share_to_drive/text_classification_model_comparison/utils/checkpoint_data/checkpoint_sales_files_embedding.pkl


In [112]:
df = pd.read_pickle(dataset_path)

print(df.columns, '\n')
df.head()

Index(['sql_sql_name', 'contact_us_form_description', 'target', 'status',
       'sales_type', 'ft', 'input_pii_removed', 'text_len',
       'language_iso639_1', 'language', 'nltk_tokenized', 'num_tokens',
       'embeddings', 'bert_based_embeddings', 'xlm_roberta_embeddings'],
      dtype='object') 



,sql_sql_name,contact_us_form_description,target,status,sales_type,ft,input_pii_removed,text_len,language_iso639_1,language,nltk_tokenized,num_tokens,embeddings,bert_based_embeddings,xlm_roberta_embeddings
0,A-270959,Looking for an individual plan,support,Qualified,Self Service,NaN,Looking for an individual plan,30,en,English,"[individual, plan]",2,"[-0.38701433, 0.1578661, -0.16941527, -0.29124...","[0.20684707, 0.16516663, 0.45383996, 0.274494,...","[-0.021906804, -0.02679788, 0.08908692, -0.004..."
1,A-270963,I'd like to know if you integrate with Talisma...,sales,Qualified,Scale,NaN,I'd like to know if you integrate with Talisma...,116,en,English,"[integrate, talisman, crm, set, asap]",5,"[-0.5965658, -0.43355212, 0.19410376, -0.11731...","[0.047534026, -0.11715461, -0.16869079, 0.0983...","[-0.031946268, -0.018111331, 0.1367632, 0.0195..."
2,A-270966,Hello - would like to learn how Lusha can help...,sales,Unqualified,NaN,NaN,Hello - would like to learn how Lusha can help...,133,en,English,"[learn, lusha, sales, team, leverage, sales, n...",9,"[-0.15919332, -0.5191771, -0.20935886, -0.5327...","[-0.19207695, -0.061742973, 0.18255222, 0.1014...","[-0.070657, -0.0052590887, 0.22000055, 0.07287..."
3,A-270968,I want to know all your plans and pricing and ...,sales,Unqualified,NaN,NaN,I want to know all your plans and pricing and ...,122,en,English,"[plans, pricing, benefits, monthly, basis, sen...",8,"[-0.56908274, -0.34940168, 0.036727045, -0.173...","[0.36898538, -0.025350137, -0.0860117, -0.0582...","[-0.016302777, -0.0058536422, 0.1011779, 0.003..."
4,A-270969,Im trying to make my account but i get the err...,support,Unqualified,NaN,NaN,Im trying to make my account but i get the err...,150,en,English,"[account, error, code, domain, marked, suspici...",10,"[-0.14534955, -0.23843925, -0.13587804, 0.1417...","[-0.06293887, -0.43082377, 0.16223387, 0.03430...","[-0.02608389, 0.04559744, 0.29144728, 0.039253..."


In [8]:
system_prompt = f'''You're a sales rep at Lusha, and an expert in classifying incoming prospects.

In Lusha, we have prospects that connect by submitting a sales form, and leave notes. Many times through the form notes, it's possible to know how to route the prospect internally.

If a prospect wants to follow up on the offer or is genuinely interested, or wants to substantially upgrade their account such as adding multiple users \ seats e.g “I want to add 3 users to my account” and not “I want to add a user to my account” etc.(as having more than two users can indicate on a potential enterprise lead), or massively increase credits, or asks about meeting or suggests time to meet, or is interested in a proof of concept or a trial, or they inquire about intent\ technographic \ job-change filters, Integrations, API features, or ask about\mention a competitor, it should probably go to the sales team.

On the other hand, if the prospect is having an issue with the product (e.g. something not working, bugs, payment issues etc.) or has a general question like "what's a credit?" etc., or is asking about a pro\professional monthly plan, or wants to add one user to their account (e.g. I want to add a user to my account), it's probably more relevant for support.

Please note: If the input says something that is entirely irrelevant, or the prospect is trying to promote\sell\market\advertise\offer-services to us\you it's irrelevant for our sales and support reps (e.g. "I'd love to discuss how your team can leverage LinkedIn Navigator and Insights to complement your current efforts and exceed your revenue targets. Would you be available for a quick chat next week?" etc.) . Same case if the message is a generic system message (e.g. DMARC report, cooking advice, or generally completely off topic.
This is true in cases where the form notes don’t provide any additional context or clear intent.

Use your best judgment as a sales representative.

These are the rules:
  - If the prospect's input indicates it's relevant for the sales team, reply: "sales"
  - If the prospect's input indicates the input is irrelevant, reply: "irrelevant"
  - If the prospect's input indicates they are trying to promote or market to us, reply: "support"
  - In any other case, reply: "support"

Please respond in one word based on the rules provided above. Do not provide an explanation or additional context.'''

# GPT-4-TURBO Model Results

In [ ]:
IF( 
  !ISBLANK({!$Record.Related_SQL__c}) && 
  OR (
     ISNEW(),
     AND (
       ISCHANGED({!$Record.StageName}),
     OR(
        ISPICKVAL({!$Record.StageName}, "Meeting Scheduled"),
        ISPICKVAL({!$Record.StageName}, "Business Discovery"),
        ISPICKVAL({!$Record.StageName}, "Closed Won"),
        ISPICKVAL({!$Record.StageName}, "Closed Lost"),
        ISPICKVAL({!$Record.StageName}, "Rejected by AE")
        )
    )
)
,TRUE, FALSE)

In [72]:
get_model_cross_validation(df, 'target', 'gpt_decision')

{'accuracy': 0.8458106098165592,
 'f1_macro': 0.7026327563331174,
 'f1_micro': 0.8458106098165592,
 'f1_weighted': 0.8690878455900922,
 'precision_macro': 0.6238348766883687,
 'precision_micro': 0.8458106098165592,
 'precision_weighted': 0.9235202644615773,
 'recall_macro': 0.9049821409246412,
 'recall_micro': 0.8458106098165592,
 'recall_weighted': 0.8458106098165592}

# GPT-4o Model

In [76]:
# Create a new batch processing file on the error set
gpt_4o_file_name = 'gpt_4o_test_set'

gpt_4o_batch_data = get_batch_completion_tasks(
    df=df,
    user_prompt_col='input_pii_removed',
    model='gpt-4o-2024-08-06',
    temperature=0,
    system_prompt=system_prompt,
    max_tokens=3
)

generate_jsonl_file(gpt_4o_file_name, gpt_4o_batch_data)

# Upload batch set to OpenAI
gpt_4o_batch_file_id = upload_file_to_openai(f'{gpt_4o_file_name}.jsonl', 'batch')
print(gpt_4o_batch_file_id)

# Create Batch Job for Error File
gpt_4o_batch_job = create_batch_completion_job(
    gpt_4o_batch_file_id,
    "/v1/chat/completions",
    {"description": f"Get GPT-4o model results on test set batch file {gpt_4o_batch_file_id}"}
)
gpt_4o_batch_job_id = gpt_4o_batch_job.id
print(gpt_4o_batch_job_id)

# Monitor Batch job Status
batch_job_output_file_id = None
batch_error_file_id      = None
job_status = None

while job_status not in ['completed', 'failed', 'cancelled']:
    # Check status every 5 minutes
    time.sleep(60*5)
    
    batch_job_status         = get_batch_job_status(gpt_4o_batch_job_id)
    job_status               = batch_job_status.status
    batch_job_output_file_id = batch_job_status.output_file_id
    batch_error_file_id      = batch_job_status.error_file_id

    print(batch_job_status, '\n')

# Save and map results back to original DF. 
gpt_4o_results_file_name = 'gpt_4o_model_results_records'
output_file = get_batch_job_results(batch_job_output_file_id)
save_batch_results_to_jsonl(output_file, gpt_4o_results_file_name)

file-HX25F5lVeWVRs7RD7tijVdc2
batch_0yx5NkQZE3eDvHqWe312nAY7
Batch(id='batch_0yx5NkQZE3eDvHqWe312nAY7', completion_window='24h', created_at=1726935354, endpoint='/v1/chat/completions', input_file_id='file-HX25F5lVeWVRs7RD7tijVdc2', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727021754, failed_at=None, finalizing_at=None, in_progress_at=1726935359, metadata={'description': 'Get GPT-4o model results on test set batch file file-HX25F5lVeWVRs7RD7tijVdc2'}, output_file_id=None, request_counts=BatchRequestCounts(completed=2956, failed=0, total=8068), _request_id='req_b8639ddd9a08328620715d570d78d676', __exclude_fields__={'_request_id', '__exclude_fields__'}) 

Batch(id='batch_0yx5NkQZE3eDvHqWe312nAY7', completion_window='24h', created_at=1726935354, endpoint='/v1/chat/completions', input_file_id='file-HX25F5lVeWVRs7RD7tijVdc2', object='batch', status='in_progress', cancelled_at=N

In [78]:
df['gpt4o_results'] = None
map_jsonl_batch_completion_results_to_df(df, gpt_4o_results_file_name, 'gpt4o_results')

In [79]:
get_model_cross_validation(df, 'target', 'gpt4o_results')

{'accuracy': 0.6116757560733763,
 'f1_macro': 0.47808285418673085,
 'f1_micro': 0.6116757560733763,
 'f1_weighted': 0.6859130636798886,
 'precision_macro': 0.47143432870486995,
 'precision_micro': 0.6116757560733763,
 'precision_weighted': 0.8862940958404248,
 'recall_macro': 0.7673629633669874,
 'recall_micro': 0.6116757560733763,
 'recall_weighted': 0.6116757560733763}

# Fine Tuning Model

#### Important Note
At this point in time, only gpt-4o is available for Fine tuning. Thus we'll try to compare the results of a fine tuned newer model of GPT-4 to the base model of gpt-4-turbo.

##### Create the fine tuning file for upload

In [11]:
fine_tuning_file_name = 'sales_form_classification_fine_tuning_file'

generate_openai_fine_tuning_json_from_df(
    fine_tuning_file_name = fine_tuning_file_name, 
    df = df, 
    system_prompt = system_prompt, 
    user_promt_col = 'input_pii_removed', 
    target_col = 'target', 
    samples_per_class = 100, 
    random_state = 12345
)

##### upload file to OpenAI

In [12]:
file_id = upload_file_to_openai(f'{fine_tuning_file_name}.jsonl', 'fine-tune')
print(file_id)

file-VHKBRDiDpLqFZ9UjgwZGZhzc


##### Create the fine tuning job in OpenAI

In [13]:
fine_tuning_job_id = fine_tune_gpt_model(
    file_id = file_id, 
    model = 'gpt-4o-2024-08-06',
    hyperparameters = {'n_epochs': 20}
)
print(fine_tuning_job_id)

ftjob-Y9UFGFjh5izTA0aTBf001aVR


##### Check Job Status Until Complete & Get Model Name

In [35]:
import time

job_id = fine_tuning_job_id
job_runnig    = True
train_loss    = None
train_mean_token_accuracy = None
attempt = 0

while job_runnig:
    if attempt > 10:
        raise Exception('ERROR: counld NOT get job stopping point!')
    else:
        job_status_events = check_fine_tuning_job_status(job_id, 10).data
        job_status_events = sorted(job_status_events, key=lambda job_status_events: job_status_events.created_at)
    
        for event in job_status_events:
            print(event, '\n\n')
            # Record model training metrics
            if hasattr(event, 'data'):
                data = event.data
                train_loss = data.get('train_loss')
                train_mean_token_accuracy = data.get('train_mean_token_accuracy')
    
            # Check if the FT job is done
            if event.message == "The job has successfully completed":
                print('Fine-tuning job completed successfuly.')
                job_running = False
    
    if not job_running: break
    attempt += 1
    time.sleep(60*10)

FineTuningJobEvent(id='ftevent-nmzi52txAyFZSi1nDBuRCgqF', created_at=1726828920, level='info', message='Step 1498/1500: training loss=0.00', object='fine_tuning.job.event', data={'step': 1498, 'train_loss': 6.5394810917496216e-06, 'total_steps': 1500, 'train_mean_token_accuracy': 1.0}, type='metrics') 


FineTuningJobEvent(id='ftevent-eHObMFpzSLxP9cRUHwqPaFIo', created_at=1726828924, level='info', message='Step 1499/1500: training loss=0.00', object='fine_tuning.job.event', data={'step': 1499, 'train_loss': 1.1444091796875e-05, 'total_steps': 1500, 'train_mean_token_accuracy': 1.0}, type='metrics') 


FineTuningJobEvent(id='ftevent-xhUNZesnSzIFCQwMpAVJQPIc', created_at=1726828940, level='info', message='Step 1500/1500: training loss=0.00', object='fine_tuning.job.event', data={'step': 1500, 'train_loss': 1.605351826583501e-05, 'total_steps': 1500, 'train_mean_token_accuracy': 1.0}, type='metrics') 


FineTuningJobEvent(id='ftevent-kqh8khtc3zgdKs97GVNmKuFW', created_at=1726828948, level

##### Get the Fine Tuned models details

In [51]:
fine_tuned_model_details = get_fine_tuned_model_details(job_id)
fine_tuned_model_name = fine_tuned_model_details.fine_tuned_model

print(f'Fine Tuned Model Name: {fine_tuned_model_name}', '\n')
print('Fine Tuned Model Name Details:')
pprint(fine_tuned_model_details.__dict__)

Fine Tuned Model Name: ft:gpt-4o-2024-08-06:personal::A9VNkczw 

Fine Tuned Model Name Details:
{'created_at': 1726826103,
 'error': Error(code=None, message=None, param=None),
 'estimated_finish': None,
 'fine_tuned_model': 'ft:gpt-4o-2024-08-06:personal::A9VNkczw',
 'finished_at': 1726828946,
 'hyperparameters': Hyperparameters(n_epochs=20, batch_size=4, learning_rate_multiplier=2),
 'id': 'ftjob-Y9UFGFjh5izTA0aTBf001aVR',
 'integrations': [],
 'model': 'gpt-4o-2024-08-06',
 'object': 'fine_tuning.job',
 'organization_id': 'org-z3Tqp7IgKTNbmvwBUDk1A5Po',
 'result_files': ['file-7jxWG94ZwYc1Ou3YygDOsInA'],
 'seed': 85812482,
 'status': 'succeeded',
 'trained_tokens': 3286640,
 'training_file': 'file-VHKBRDiDpLqFZ9UjgwZGZhzc',
 'user_provided_suffix': None,
 'validation_file': None}


##### Create test file for batch job in OpenAI

In [60]:
test_set_file_name = 'fine_tune_test_set'

batch_data = get_batch_completion_tasks(
    df=df,
    user_prompt_col='input_pii_removed',
    model=fine_tuned_model_name,
    temperature=0,
    system_prompt=system_prompt,
    max_tokens=3
)

generate_jsonl_file(test_set_file_name, batch_data)

##### Upload file to OpenAI for batch completion job

In [64]:
test_set_file_id = upload_file_to_openai(f'{test_set_file_name}.jsonl', 'batch')
print(test_set_file_id)

'file-6wThQtTVGKBnVsvptuQ7wCy4'

In [14]:
batch_job = create_batch_completion_job(
    test_set_file_id,
    "/v1/chat/completions",
    {"description": f"Get FT model {fine_tuned_model_name} results on test set batch file {test_set_file_id}"}
)
batch_job_id = batch_job.id
batch_job

Batch(id='batch_d3QuDz8Ietb3s4mWFlNVdZCU', completion_window='24h', created_at=1726912088, endpoint='/v1/chat/completions', input_file_id='file-6wThQtTVGKBnVsvptuQ7wCy4', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1726998488, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Get FT model ft:gpt-4o-2024-08-06:personal::A9VNkczw results on test set batch file file-6wThQtTVGKBnVsvptuQ7wCy4'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0), _request_id='req_85d231db59a2a1ac48be8ee4db38bf11', __exclude_fields__={'_request_id', '__exclude_fields__'})

In [24]:
batch_job_output_file_id = None
batch_error_file_id      = None
job_status = None

while job_status not in ['completed', 'failed', 'cancelled']:
    # Check status every 5 minutes
    time.sleep(60*5)
    
    batch_job_status         = get_batch_job_status(batch_job_id)
    job_status               = batch_job_status.status
    batch_job_output_file_id = batch_job_status.output_file_id
    batch_error_file_id      = batch_job_status.error_file_id

    print(batch_job_status, '\n')

Batch(id='batch_d3QuDz8Ietb3s4mWFlNVdZCU', completion_window='24h', created_at=1726912088, endpoint='/v1/chat/completions', input_file_id='file-6wThQtTVGKBnVsvptuQ7wCy4', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1726998488, failed_at=None, finalizing_at=None, in_progress_at=1726912093, metadata={'description': 'Get FT model ft:gpt-4o-2024-08-06:personal::A9VNkczw results on test set batch file file-6wThQtTVGKBnVsvptuQ7wCy4'}, output_file_id=None, request_counts=BatchRequestCounts(completed=5341, failed=2657, total=8068), _request_id='req_2bf02405f651f624da0caedb4fded0a8', __exclude_fields__={'_request_id', '__exclude_fields__'}) 

Batch(id='batch_d3QuDz8Ietb3s4mWFlNVdZCU', completion_window='24h', created_at=1726912088, endpoint='/v1/chat/completions', input_file_id='file-6wThQtTVGKBnVsvptuQ7wCy4', object='batch', status='in_progress', cancelled_at=None, cancelling_at=Non

In [44]:
output_file = get_batch_job_results(batch_job_output_file_id)
save_batch_results_to_jsonl(output_file, 'gpt_4o_ft_model_results')

df['gpt4_fine_tuned_results'] = None
map_jsonl_batch_completion_results_to_df(df, 'gpt_4o_ft_model_results', 'gpt4_fine_tuned_results')

In [45]:
df['gpt4_fine_tuned_results'].value_counts()

gpt4_fine_tuned_results
sales         3779
support       1082
irrelevant     479
1. support       1
Name: count, dtype: int64

In [51]:
if batch_error_file_id:
    # Generate Error file for inspection
    ft_model_batch_errors = 'gpt_4o_ft_model_erros'
    output_file = get_batch_job_results(batch_error_file_id)
    save_batch_results_to_jsonl(output_file, ft_model_batch_errors)    

In [55]:
### Retry error rows Assuming the errors are 429's (otherwise comment out) ###
# Rerun process as many times as needed.

# filter all records that need to be retried due to error in batch processing.
err_records = set({})

with open(f'{ft_model_batch_errors}.jsonl', 'r') as f:
    for line in f:
        row = json.loads(line)
        if row.get('custom_id', False):
            err_records.add(int(row['custom_id']))
            
df_erros = df.loc[df.index.intersection(err_records)]

# Create a new batch processing file on the error set
err_set_file_name = 'err_fine_tune_test_set'

err_batch_data = get_batch_completion_tasks(
    df=df_erros,
    user_prompt_col='input_pii_removed',
    model=fine_tuned_model_name,
    temperature=0,
    system_prompt=system_prompt,
    max_tokens=3
)

generate_jsonl_file(err_set_file_name, err_batch_data)

# Upload error batch set to OpenAI
err_test_set_file_id = upload_file_to_openai(f'{err_set_file_name}.jsonl', 'batch')
print(err_test_set_file_id)

# Create Batch Job for Error File
err_batch_job = create_batch_completion_job(
    err_test_set_file_id,
    "/v1/chat/completions",
    {"description": f"Get FT model {fine_tuned_model_name} results on test set batch file {err_test_set_file_id}"}
)
err_batch_job_id = err_batch_job.id
print(err_batch_job)

# Monitor Batch job Status
batch_job_output_file_id = None
batch_error_file_id      = None
job_status = None

while job_status not in ['completed', 'failed', 'cancelled']:
    # Check status every 5 minutes
    time.sleep(60*5)
    
    batch_job_status         = get_batch_job_status(err_batch_job_id)
    job_status               = batch_job_status.status
    batch_job_output_file_id = batch_job_status.output_file_id
    batch_error_file_id      = batch_job_status.error_file_id

    print(batch_job_status, '\n')

# Save and map results back to original DF. 
err_file_ft_results_name = 'gpt_4o_ft_model_results_err_records'
output_file = get_batch_job_results(batch_job_output_file_id)
save_batch_results_to_jsonl(output_file, err_file_ft_results_name)

file-zI8ON2j09af4bYUEi5Kx0AHD
Batch(id='batch_sAQ9ecF4xcgXtcDGLYxmeQGR', completion_window='24h', created_at=1726929310, endpoint='/v1/chat/completions', input_file_id='file-zI8ON2j09af4bYUEi5Kx0AHD', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727015710, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Get FT model ft:gpt-4o-2024-08-06:personal::A9VNkczw results on test set batch file file-zI8ON2j09af4bYUEi5Kx0AHD'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0), _request_id='req_57587d90cf7e232048d09d6774f128d1', __exclude_fields__={'_request_id', '__exclude_fields__'})
Batch(id='batch_sAQ9ecF4xcgXtcDGLYxmeQGR', completion_window='24h', created_at=1726929310, endpoint='/v1/chat/completions', input_file_id='file-zI8ON2j09af4bYUEi5Kx0AHD', object='batch', status='in_progress', cancelled_at=None, cance

NameError: name 'map_jsonl_batch_completion_results_to_df' is not defined

In [57]:
map_jsonl_batch_completion_results_to_df(df, err_file_ft_results_name, 'gpt4_fine_tuned_results')

In [60]:
df['gpt4_fine_tuned_results'].value_counts()

gpt4_fine_tuned_results
sales         5781
support       1643
irrelevant     632
1. support       1
Name: count, dtype: int64

In [64]:
def normalize_prediction(pred: str) -> str:
    valid_preds = set({'sales', 'support', 'irrelevant'})

    if not pred or pred in valid_preds: return pred
    elif 'sale'       in pred:          return 'sales'
    elif 'support'    in pred:          return 'support'    
    elif 'irrelevant' in pred:          return 'irrelevant'
    else:                               return pred
        

In [65]:
df["gpt4_fine_tuned_results"] = df["gpt4_fine_tuned_results"].apply(normalize_prediction)

In [66]:
df['gpt4_fine_tuned_results'].value_counts()

gpt4_fine_tuned_results
sales         5781
support       1644
irrelevant     632
Name: count, dtype: int64

In [70]:
df_ft_results = df[~df['gpt4_fine_tuned_results'].isna()]
get_model_cross_validation(df_ft_results,'target', 'gpt4_fine_tuned_results')

{'accuracy': 0.7762194365148318,
 'f1_macro': 0.5782443612427958,
 'f1_micro': 0.7762194365148318,
 'f1_weighted': 0.8143054276079603,
 'precision_macro': 0.520910951512416,
 'precision_micro': 0.7762194365148318,
 'precision_weighted': 0.8930905775680181,
 'recall_macro': 0.8058122850474941,
 'recall_micro': 0.7762194365148318,
 'recall_weighted': 0.7762194365148318}

In [80]:
df.to_csv('sales_inbound_with_gpt_predictions.csv')